# PseudoKnowYourMemes

Connect drive, to be able to use the data easily.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We are going to mimic the notebook from class to classify memes based on their meme format. For example this one is a random Di Caprio format meme: https://i.kym-cdn.com/photos/images/newsfeed/001/887/154/fab.jpg

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
img_height=299
img_width=299

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(115, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', metrics=['accuracy'],loss='categorical_crossentropy')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./drive/MyDrive/PseudoKnowYourMemes/train/train', target_size=(img_height, img_width), batch_size=20, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory('./drive/MyDrive/PseudoKnowYourMemes/valid/valid', target_size=(img_height, img_width), batch_size=20, class_mode='categorical')

Found 1143 images belonging to 115 classes.
Found 4573 images belonging to 115 classes.


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=50,validation_data=validation_generator,validation_steps=10,epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
50/50 [==============================] - 310s 6s/step - loss: 4.0470 - accuracy: 0.2543 - val_loss: 2.2540 - val_accuracy: 0.6700
Epoch 2/3
50/50 [==============================] - 103s 2s/step - loss: 1.2068 - accuracy: 0.8026 - val_loss: 0.8611 - val_accuracy: 0.8700
Epoch 3/3
50/50 [==============================] - 84s 2s/step - loss: 0.4152 - accuracy: 0.9349 - val_loss: 0.2905 - val_accuracy: 0.9450


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=50,validation_data=validation_generator,validation_steps=10,epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
50/50 [==============================] - 75s 2s/step - loss: 0.2068 - accuracy: 0.9736 - val_loss: 0.3161 - val_accuracy: 0.9400
Epoch 2/3
50/50 [==============================] - 74s 1s/step - loss: 0.1223 - accuracy: 0.9888 - val_loss: 0.2135 - val_accuracy: 0.9700
Epoch 3/3
50/50 [==============================] - 70s 1s/step - loss: 0.0918 - accuracy: 0.9908 - val_loss: 0.1996 - val_accuracy: 0.9650


Just a little more training

In [ ]:
model.fit_generator(train_generator,steps_per_epoch=50,validation_data=validation_generator,validation_steps=10,epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
50/50 [==============================] - 27s 536ms/step - loss: 0.1772 - accuracy: 0.9383 - val_loss: 0.2281 - val_accuracy: 0.9400


Let's calculate the accuracy on the test set aswell.

In [ ]:
test_err = model.evaluate(test_generator)
print("Loss on test set:", test_err[0], "Accuracy on test set:", test_err[1])

15/15 [==============================] - 6s 395ms/step - loss: 0.1599 - accuracy: 0.9533
Loss on test set: 0.15985926985740662 Accuracy on test set: 0.95333331823349


We can see that the accuracy on the train set is 93.8% on the validation set 94% and on the test set 95.3%.

Now unfreeze some layers of the base_model, and train further the neural network with some small learning rate sgd.

In [ ]:
for layer in model.layers[:280]:
       layer.trainable = False
for layer in model.layers[280:]:
       layer.trainable = True

In [ ]:
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), metrics=['accuracy'], loss='categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=20,validation_data=validation_generator,validation_steps=10,epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
20/20 [==============================] - 27s 1s/step - loss: 0.1785 - accuracy: 0.9350 - val_loss: 0.2260 - val_accuracy: 0.9300
Epoch 2/3
20/20 [==============================] - 20s 1s/step - loss: 0.1677 - accuracy: 0.9475 - val_loss: 0.2199 - val_accuracy: 0.9400
Epoch 3/3
20/20 [==============================] - 20s 1000ms/step - loss: 0.1628 - accuracy: 0.9450 - val_loss: 0.1979 - val_accuracy: 0.9450


In [ ]:
test_err = model.evaluate(test_generator)
print("Loss on test set:", test_err[0], "Accuracy on test set:", test_err[1])

15/15 [==============================] - 6s 391ms/step - loss: 0.1643 - accuracy: 0.9467
Loss on test set: 0.16426116228103638 Accuracy on test set: 0.9466666579246521


We can see that the accuracy on the train set is 94.5% on the validation set 94.5% and on the test set 94.7%.